# 1

In [12]:
import gurobipy as gp
from gurobipy import GRB

def solve_transportation_problem(factory_supply, warehouse_demand, route_expenses):
    # Create a Gurobi model
    model = gp.Model("TransportationProblem")

    # Decision variables
    x = {}
    for i in range(len(factory_supply)):
        for j in range(len(warehouse_demand)):
            x[i, j] = model.addVar(vtype=GRB.CONTINUOUS, name=f"x_{i}_{j}")

    # Objective function: Minimize total expenses
    model.setObjective(gp.quicksum(route_expenses[i][j] * x[i, j] for i in range(len(factory_supply))
                                   for j in range(len(warehouse_demand))), GRB.MINIMIZE)

    # Constraints: Demand constraints
    for j in range(len(warehouse_demand)):
        model.addConstr(gp.quicksum(x[i, j] for i in range(len(factory_supply))) == warehouse_demand[j],
                        f"demand_{j}")

    # Constraints: Production limits
    for i in range(len(factory_supply)):
        model.addConstr(gp.quicksum(x[i, j] for j in range(len(warehouse_demand))) <= factory_supply[i],
                        f"production_{i}")

    # Solve the model
    model.optimize()

    # Check if the optimization was successful
    if model.status == GRB.OPTIMAL:
        # Display the optimal solution
        print("\nOptimal Solution:")
        for i in range(len(factory_supply)):
            for j in range(len(warehouse_demand)):
                if x[i, j].x > 0:
                    print(f"Factory {i} to Warehouse {j}: {x[i, j].x} units")

        # Display objective value
        print(f"\nTotal Expenses: ${model.objVal}")

        # Sensitivity Analysis for Objective Coefficient Changes
        print("\nSensitivity Analysis for Objective Coefficient Changes:")
        for v in model.getVars():
            print(f"{v.VarName}: Reduced Cost = {v.RC}")

        # Sensitivity Analysis for Right-Hand Side Changes (Demand)
        print("\nSensitivity Analysis for Right-Hand Side (Demand) Changes:")
        for j in range(len(warehouse_demand)):
            demand_constr = model.getConstrByName(f"demand_{j}")
            print(f"Change in demand for Warehouse {j}:")
            for change in [-5000, -2000, 2000, 5000]:
                # Change right-hand side of the demand constraint
                demand_constr.setAttr(GRB.Attr.RHS, warehouse_demand[j] + change)

                # Resolve the model with the updated demand
                model.optimize()

                # Display the objective value for the updated demand
                print(f"  New Total Expenses (Demand +{change}): ${model.objVal}")

            # Reset the demand constraint to its original RHS
            demand_constr.setAttr(GRB.Attr.RHS, warehouse_demand[j])

    else:
        print("Optimization was not successful. Check the model and data.")

In [13]:
# Example data
factory_supply = [150000, 200000]  # Production limits for factories
warehouse_demand = [70000, 50000, 100000, 40000]  # Demand for warehouses
route_expenses = [
    [15, 15, 30, 6],  # Expense from Factory 0 to Warehouses 0, 1, 2
    [9, 9, 15, 6],  # Expense from Factory 1 to Warehouses 0, 1, 2
]

# Solve the transportation problem and perform sensitivity analysis
solve_transportation_problem(factory_supply, warehouse_demand, route_expenses)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 8 columns and 16 nonzeros
Model fingerprint: 0x2a116410
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 2e+05]
Presolve removed 5 rows and 5 columns
Presolve time: 0.02s
Presolved: 1 rows, 4 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0400000e+06   1.250000e+04   0.000000e+00      0s
       1    2.9400000e+06   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.04 seconds (0.00 work units)
Optimal objective  2.940000000e+06

Optimal Solution:
Factory 0 to Warehouse 0: 20000.0 units
Factory 0 to Warehouse 3: 40000.0 units
Factory 1 to Warehouse 0: 50000.0 units
Factory

  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 2e+05]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8980000e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds (0.00 work units)
Optimal objective  2.898000000e+06
  New Total Expenses (Demand +-2000): $2898000.0
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 8 columns and 16 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 2e+05]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9820000e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 